In [13]:
import algorithm as al
import numpy as np
import pandas as pd

In [14]:
# train = pd.read_excel('../archive/Raisin_Dataset.xlsx')
train = pd.read_csv('../data/breast+cancer+coimbra/dataR2.csv', header=None)
train

,0,1,2,3,4,5,6,7,8,9
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1
...,...,...,...,...,...,...,...,...,...,...
111,45,26.850000,92,3.330,0.755688,54.6800,12.100000,10.96000,268.230,2
112,62,26.840000,100,4.530,1.117400,12.4500,21.420000,7.32000,330.160,2
113,65,32.050000,97,5.730,1.370998,61.4800,22.540000,10.33000,314.050,2
114,72,25.590000,82,2.820,0.570392,24.9600,33.750000,3.27000,392.460,2


In [15]:
train = train.to_numpy()
R_train = train[:, 0:-1]
R_train
# R_train = np.random.rand(100, 10)

array([[ 48.        ,  23.5       ,  70.        , ...,   9.7024    ,
          7.99585   , 417.114     ],
       [ 83.        ,  20.69049454,  92.        , ...,   5.429285  ,
          4.06405   , 468.786     ],
       [ 82.        ,  23.12467037,  91.        , ...,  22.43204   ,
          9.27715   , 554.697     ],
       ...,
       [ 65.        ,  32.05      ,  97.        , ...,  22.54      ,
         10.33      , 314.05      ],
       [ 72.        ,  25.59      ,  82.        , ...,  33.75      ,
          3.27      , 392.46      ],
       [ 86.        ,  27.18      , 138.        , ...,  14.11      ,
          4.35      ,  90.09      ]])

In [16]:
M, N, H = al.method1(R_train, 0.45)

M = np.around(M, 3)
N = np.around(N, 3)
H = np.around(H, 3)

print ("M = ",M)
print ("\nN = ",N)
print ("\nH = ",H)

M =  [[0.024 0.025 0.041 ... 0.056 0.074 0.039]
 [0.023 0.021 0.037 ... 0.044 0.064 0.034]
 [0.023 0.022 0.037 ... 0.049 0.064 0.034]
 ...
 [0.021 0.024 0.037 ... 0.046 0.065 0.033]
 [0.019 0.02  0.032 ... 0.045 0.056 0.029]
 [0.016 0.017 0.029 ... 0.032 0.05  0.024]]

N =  [[0.631 0.624 0.547 ... 0.493 0.438 0.555]
 [0.639 0.648 0.566 ... 0.534 0.468 0.58 ]
 [0.64  0.644 0.564 ... 0.515 0.466 0.578]
 ...
 [0.653 0.63  0.565 ... 0.528 0.463 0.584]
 [0.667 0.66  0.587 ... 0.529 0.493 0.602]
 [0.69  0.678 0.605 ... 0.589 0.514 0.63 ]]

H =  [[0.344 0.351 0.412 ... 0.452 0.488 0.406]
 [0.338 0.331 0.397 ... 0.421 0.468 0.387]
 [0.337 0.334 0.399 ... 0.436 0.47  0.388]
 ...
 [0.326 0.346 0.398 ... 0.426 0.472 0.383]
 [0.314 0.32  0.381 ... 0.425 0.452 0.369]
 [0.294 0.305 0.366 ... 0.379 0.437 0.346]]


In [17]:
# Chuẩn bị dữ liệu cho phân cụm
X = np.dstack((M, N, H))
P = X.shape[0]
true_labels = train[:, -1]

# Danh sách các hàm độ đo khoảng cách
distance_measures = {
    'distance_function': al.distance_function,
    'hamming_distance': al.hamming_distance,
    'euclidean_distance': al.euclidean_distance,
    'normalized_euclidean_distance': al.normalized_euclidean_distance,
    'hausdorff_distance': al.hausdorff_distance,
    'yang_chiclana_distance': al.yang_chiclana_distance,
    'wang_xin_distance': al.wang_xin_distance,
    'liu_jiang_distance': al.liu_jiang_distance,
    'he_distance': al.he_distance,
    'thao_distance': al.thao_distance,
    'mahanta_panda_distance': al.mahanta_panda_distance,
}

# Vòng lặp qua các độ đo
for measure_name, measure_func in distance_measures.items():
    # Phân cụm sử dụng độ đo hiện tại
    U, S = al.method3(R_train, len(np.unique(train[:, -1])), 1.7, 0.45, measure_name)

    # Gán nhãn dựa trên khoảng cách tới các tâm cụm
    y = np.array([np.argmin([measure_func(X[i], S[j]) for j in range(S.shape[0])]) for i in range(P)])

    # Tính toán các chỉ số đánh giá
    CA = round(al.calculate_CA(true_labels, y), 2)
    PC = round(al.calculate_PC(U), 2)
    SC = round(al.calculate_SC(X, U, S, 1.7), 2)
    XB = round(al.calculate_XB(X, U, S), 2)
    DI = round(al.calculate_DI(X, U), 2)

    # In ra các chỉ số đánh giá cho mỗi độ đo
    print(f'Measure: {measure_name}')
    print(f'  Clustering Accuracy (CA): {CA}')
    print(f'  Partition Coefficient (PC): {PC}')
    print(f'  Partition Index (SC): {SC}')
    print(f'  Xie-Beni Index (XB): {XB}')
    print(f'  Dunn Index (DI): {DI}')
    print('------------------------------------------------')

Measure: distance_function
  Clustering Accuracy (CA): 0.37
  Partition Coefficient (PC): 0.88
  Partition Index (SC): 0.12
  Xie-Beni Index (XB): 1.42
  Dunn Index (DI): 0.05
------------------------------------------------
Measure: hamming_distance
  Clustering Accuracy (CA): 0.27
  Partition Coefficient (PC): 0.68
  Partition Index (SC): 0.4
  Xie-Beni Index (XB): 3.18
  Dunn Index (DI): 0.04
------------------------------------------------
Measure: euclidean_distance
  Clustering Accuracy (CA): 0.17
  Partition Coefficient (PC): 0.65
  Partition Index (SC): 0.46
  Xie-Beni Index (XB): 3.53
  Dunn Index (DI): 0.04
------------------------------------------------
Measure: normalized_euclidean_distance
  Clustering Accuracy (CA): 0.17
  Partition Coefficient (PC): 0.65
  Partition Index (SC): 0.46
  Xie-Beni Index (XB): 3.53
  Dunn Index (DI): 0.04
------------------------------------------------
Measure: hausdorff_distance
  Clustering Accuracy (CA): 0.15
  Partition Coefficient (PC)

In [18]:
M, N, H = al.method2_1(R_train, 0, 1, 0.45, 1)

M = np.around(M, 3)
N = np.around(N, 3)
H = np.around(H, 3)

# Chuẩn bị dữ liệu cho phân cụm
X = np.dstack((M, N, H))
P = X.shape[0]
true_labels = train[:, -1]

# Danh sách các hàm độ đo khoảng cách
distance_measures = {
    'distance_function': al.distance_function,
    'hamming_distance': al.hamming_distance,
    'euclidean_distance': al.euclidean_distance,
    'normalized_euclidean_distance': al.normalized_euclidean_distance,
    'hausdorff_distance': al.hausdorff_distance,
    'yang_chiclana_distance': al.yang_chiclana_distance,
    'wang_xin_distance': al.wang_xin_distance,
    'liu_jiang_distance': al.liu_jiang_distance,
    'he_distance': al.he_distance,
    'thao_distance': al.thao_distance,
    'mahanta_panda_distance': al.mahanta_panda_distance,
}

# Vòng lặp qua các độ đo
for measure_name, measure_func in distance_measures.items():
    # Phân cụm sử dụng độ đo hiện tại
    U, S = al.method3_1(R_train, len(np.unique(train[:, -1])), 1.7, 0.45, measure_name)

    # Gán nhãn dựa trên khoảng cách tới các tâm cụm
    y = np.array([np.argmin([measure_func(X[i], S[j]) for j in range(S.shape[0])]) for i in range(P)])

    # Tính toán các chỉ số đánh giá
    CA = round(al.calculate_CA(true_labels, y), 2)
    PC = round(al.calculate_PC(U), 2)
    SC = round(al.calculate_SC(X, U, S, 1.7), 2)
    XB = round(al.calculate_XB(X, U, S), 2)
    DI = round(al.calculate_DI(X, U), 2)

    # In ra các chỉ số đánh giá cho mỗi độ đo
    print(f'Measure: {measure_name}')
    print(f'  Clustering Accuracy (CA): {CA}')
    print(f'  Partition Coefficient (PC): {PC}')
    print(f'  Partition Index (SC): {SC}')
    print(f'  Xie-Beni Index (XB): {XB}')
    print(f'  Dunn Index (DI): {DI}')
    print('------------------------------------------------')

Measure: distance_function
  Clustering Accuracy (CA): 0.26
  Partition Coefficient (PC): 0.59
  Partition Index (SC): 1.69
  Xie-Beni Index (XB): 1.6
  Dunn Index (DI): 0.14
------------------------------------------------
Measure: hamming_distance
  Clustering Accuracy (CA): 0.29
  Partition Coefficient (PC): 0.5
  Partition Index (SC): 2.884907682555356e+18
  Xie-Beni Index (XB): 2.4872022577331697e+18
  Dunn Index (DI): 0.15
------------------------------------------------
Measure: euclidean_distance
  Clustering Accuracy (CA): 0.26
  Partition Coefficient (PC): 0.5
  Partition Index (SC): 1.0628834338749432e+18
  Xie-Beni Index (XB): 9.163572520626815e+17
  Dunn Index (DI): 0.14
------------------------------------------------
Measure: normalized_euclidean_distance
  Clustering Accuracy (CA): 0.26
  Partition Coefficient (PC): 0.5
  Partition Index (SC): 7.452308100438605e+16
  Xie-Beni Index (XB): 6.4249534377876696e+16
  Dunn Index (DI): 0.14
------------------------------------